In [ ]:

import numpy as np
import pandas as pd
from shapely.geometry import LineString, Point, shape
import geopandas as gpd
from geopy.distance import great_circle
import xarray as xr
from geographiclib.geodesic import Geodesic

# From the ipynb driver, user input: ###################################
# User Inputs:
start_time_str       = '2023-01-01T00:00:00Z'
stop_time_str        = '2023-12-31T23:59:59Z'
query_limit          = 15e4
send_notification    = True
make_plot            = True
output_dir           = "/scratch/omg28/Data/"

# Convert start and stop times to datetime objects
start_time_simple = pd.to_datetime(start_time_str).strftime("%Y-%m-%d")
stop_time_simple = pd.to_datetime(stop_time_str).strftime("%Y-%m-%d")
analysis_year = pd.to_datetime(start_time_str).year

# Define grid
lat_bins = np.arange(-90, 90.1, 0.5)
lon_bins = np.arange(-180, 180.1, 0.5)
alt_bins_ft = np.arange(0, 55001, 1000)
alt_bins_m = alt_bins_ft * 0.3048
nlat, nlon, nalt = len(lat_bins)-1, len(lon_bins)-1, len(alt_bins_m)-1
########################################################################

# MATH HELPER FUNCTIONS ##################################
def secd(theta):
    """Convert degrees to seconds."""
    if theta%360 == 90 or theta%360 == 270:
        return np.nan
    return 1/np.cos(np.radians(theta))
###########################################################


# Define conflict countries and buffer degrees

conflict_countries = ['Russia', 'Ukraine', 'Libya', 'Syria', 'Sudan', 'Yemen']
buffer_degrees = 1.0

# Load country boundaries
world = gpd.read_file('ne_110m_admin_0_countries.zip')
name_col = 'NAME' if 'NAME' in world.columns else 'name'

# Get geometries for conflict countries
conflict_geometries = []
for c in conflict_countries:
    country_geom = world[world[name_col] == c].geometry
    if not country_geom.empty:
        conflict_geometries.append(country_geom.union_all().buffer(buffer_degrees))

# Create a union of `all conflict areas
if conflict_geometries:
    conflict_areas = conflict_geometries[0]
    for geom in conflict_geometries[1:]:
        conflict_areas = conflict_areas.union(geom)
    conflict_areas_buffered = conflict_areas.buffer(buffer_degrees)
else:
    # If no valid countries found, create empty geometry
    conflict_areas_buffered = Point(0, 0).buffer(0)


#TEST CODE ##################################
flights = pd.read_pickle('/scratch/omg28/Data/2023-01-01_to_2023-01-31_labeled.pkl')
row = flights.iloc[244256] # long flight from EGLL to NZCH
era5_file = f"/scratch/omg28/Data/winddb/era5_wind_{analysis_year}.nc"
ds_era5 = xr.open_dataset(era5_file)
print(ds_era5)

cruise_alt_ft = 35000  # feet example value
cruise_speed_ms = 250  # m/s, example value
#################################################################


cruise_distance_m = row['gc_FEAT_km'] * 1000


import numpy as np
import pandas as pd
import pickle
from generate_flightpath import generate_flightpath
import os
from multiprocessing import Pool, cpu_count
from geographiclib.geodesic import Geodesic
from xgboost import XGBRegressor

SECONDS_PER_MONTH = 31 * 24 * 3600  # January
REMOVAL_TIMESCALE_S = 2 * 24 * 3600  # 2 days

def get_cruise_params(typecode, perf_df):
    try:
        cruise_alt_ft = perf_df.loc[typecode, 'cruise_Ceiling'] * 100 if not pd.isnull(perf_df.loc[typecode, 'cruise_Ceiling']) else 35000
        cruise_speed_ms = perf_df.loc[typecode, 'cruise_TAS'] * 0.514444 if not pd.isnull(perf_df.loc[typecode, 'cruise_TAS']) else 250
        if cruise_alt_ft <= 0 or np.isnan(cruise_alt_ft):
            cruise_alt_ft = 35000
        if cruise_speed_ms <= 0 or np.isnan(cruise_speed_ms):
            cruise_speed_ms = 250
        return cruise_alt_ft, cruise_speed_ms
    except Exception:
        return 35000, 250

def process_flight(args):
    row, xgb_models, perf_df, lat_bins, lon_bins, alt_bins_ft, nlat, nlon, nalt = args
    typecode = row['typecode']
    model = xgb_models.get(typecode)
    if model is None:
        return []
    try:
        cruise_alt_ft, cruise_speed_ms = get_cruise_params(typecode, perf_df)
        fp = generate_flightpath(typecode, row['gc_FEAT_km'], None)
        cruise_alt_ft = fp.get('cruise', {}).get('cruise_altitude_ft', cruise_alt_ft)
    except Exception:
        cruise_alt_ft, cruise_speed_ms = get_cruise_params(typecode, perf_df)
    features = np.array([[row['gc_FEAT_km'], cruise_alt_ft]])
    mean_nox_flux = model.predict(features)[0]
    cruise_distance_m = row['gc_FEAT_km'] * 1000
    cruise_time_s = cruise_distance_m / cruise_speed_ms
    total_nox_g = mean_nox_flux * cruise_time_s
    total_nox_kg = total_nox_g / 1000
    
    n_segments = int(np.ceil(cruise_distance_m / 10000))
    geod = Geodesic.WGS84
    line = geod.InverseLine(row['estdeparturelat'], row['estdeparturelong'],
                            row['estarrivallat'], row['estarrivallong'])
    ds = cruise_distance_m / n_segments
    lats, lons = [], []
    for i in range(n_segments):
        s = min(ds * i, line.s13)
        pos = line.Position(s)
        lats.append(pos['lat2'])
        lons.append(pos['lon2'])
    alts = np.full(n_segments, cruise_alt_ft)

    box_fraction = REMOVAL_TIMESCALE_S / (SECONDS_PER_MONTH + REMOVAL_TIMESCALE_S)
    nox_per_segment = total_nox_kg / n_segments * box_fraction

    updates = []
    for i in range(n_segments):
        lat_idx = np.searchsorted(lat_bins, lats[i], side='right') - 1
        lon_idx = np.searchsorted(lon_bins, lons[i], side='right') - 1
        alt_idx = np.searchsorted(alt_bins_ft, alts[i], side='right') - 1
        if 0 <= lat_idx < nlat and 0 <= lon_idx < nlon and 0 <= alt_idx < nalt:
            updates.append((lat_idx, lon_idx, alt_idx, nox_per_segment))       
    return updates

def process_month_emissions(
    month_start_time_str: str,
    output_dir: str = "/scratch/omg28/Data/no_track2023/emissions/",
    performance_and_emissions_model: pd.DataFrame = pd.read_pickle('performance_and_emissions_model.pkl')
):
    start_time_str_loop = pd.to_datetime(month_start_time_str)
    stop_time_str_loop = (start_time_str_loop + pd.offsets.MonthEnd(1)).replace(hour=23, minute=59, second=59)
    start_time_simple_loop = pd.to_datetime(start_time_str_loop).strftime("%Y-%m-%d")
    stop_time_simple_loop = pd.to_datetime(stop_time_str_loop).strftime("%Y-%m-%d")

    # Load flights data
    monthly_flights = pd.read_pickle(f'{output_dir}/{start_time_simple_loop}_to_{stop_time_simple_loop}_filtered.pkl')
    model_dir = 'saved_models_nox_flux'
    typecodes = monthly_flights['typecode'].unique()

    # Load all xgboost models into memory for speed
    xgb_models = {}
    for typecode in typecodes:
        model_path = os.path.join(model_dir, f'xgb_{typecode}.ubj')
        if os.path.exists(model_path):
            model = XGBRegressor()
            model.load_model(model_path)
            xgb_models[typecode] = model


    # Prepare cruise altitude and speed lookup from performance_and_emissions_model
    perf_df = performance_and_emissions_model.set_index('typecode')

    # Define grid
    lat_bins = np.arange(-90, 90.1, 0.5)
    lon_bins = np.arange(-180, 180.1, 0.5)
    alt_bins_ft = np.arange(0, 55001, 1000)
    alt_bins_m = alt_bins_ft * 0.3048
    nlat, nlon, nalt = len(lat_bins)-1, len(lon_bins)-1, len(alt_bins_m)-1
    nox_grid = np.zeros((nlat, nlon, nalt), dtype=np.float64)

    # Prepare arguments for loop
    pool_args = [
        (row, xgb_models, perf_df, lat_bins, lon_bins, alt_bins_ft, nlat, nlon, nalt)
        for _, row in monthly_flights.iterrows()
    ]

    # Simple for loop instead of multiprocessing
    results = []
    for args in pool_args:
        updates = process_flight(args)
        results.append(updates)

    # Aggregate results
    for updates in results:
        for lat_idx, lon_idx, alt_idx, nox in updates:
            nox_grid[lat_idx, lon_idx, alt_idx] += nox

    # Optionally: Save as NetCDF or CSV for further analysis
    output_dir = os.path.expanduser(output_dir)
    os.makedirs(f'{output_dir}/emissions', exist_ok=True)
    filename = os.path.join(output_dir, f'emissions/{start_time_simple_loop}_to_{stop_time_simple_loop}_NOx_nowar.npy')
    np.save(filename, nox_grid)
    return filename


In [ ]:
flights = pd.read_pickle('/scratch/omg28/Data/2023-01-01_to_2023-01-31_labeled.pkl')
flights.info()

In [ ]:
import os
import sys

proj_path = os.path.abspath(os.path.join("FlightTrajectories/FlightTrajectories"))
if proj_path not in sys.path:
    sys.path.append(proj_path)
import optim_iagos_only



ModuleNotFoundError: No module named 'FlightTrajectories.misc_geo'